# **OCI Search with OpenSearch**

In [2]:

## Create an OCI Search with Opensearch cluster in OCI, in private subnet in Chicago. Add ports
## Notebook in same private subnet as opensearch cluster
## install conda

## What are people asking? What are people searching for?

In [2]:
#!pip install langchain sentence-transformers

In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import OpenSearchVectorSearch

## **Credentials**

In [8]:
## The API endpoint, without https//
apiEndpoint = "amaaaaaangencdyav4dubl3xfd4vdxd7flcb32tothcprrq2skkhvchofbtq.opensearch.eu-frankfurt-1.oci.oraclecloud.com"

## port
port = 9200

##Credentials
username = "bobpeulen"
password = "x"

## Name of index
searchIndex = 'index_3'

## **Create and ping connection**

In [9]:
## The API endpoint, without https//
apiEndpoint = "amaaaaaangencdyav4dubl3xfd4vdxd7flcb32tothcprrq2skkhvchofbtq.opensearch.eu-frankfurt-1.oci.oraclecloud.com"

## port
port = 9200

##Credentials
username = "bobpeulen"
password = "xxxxx"

## Name of index
searchIndex = 'index_3'

auth = (username, password)

client = OpenSearch(
    hosts = [{'host': apiEndpoint, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)
client.ping()

False

## **Create a Mapping and Index**

In [7]:
index_body = {
  'settings': {
    "index.knn": True
  },
  "mappings": {
    "properties": {
      "osha_vector": {
        "type": "knn_vector",
        "dimension": 1536,
        "method": {
          "engine": "faiss",
          "name": "hnsw",
          "space_type": "l2"
        }
      }
    }
  }
}

response = client.indices.create('index_rag', body=index_body)
print(response)

InvalidURL: Failed to parse: https://[http://amaaaaaangencdyav4dubl3xfd4vdxd7flcb32tothcprrq2skkhvchofbtq.opensearch.eu-frankfurt-1.oci.oraclecloud.com]:9200/index_rag

## **See all indexes**

In [53]:
client.indices.get_alias(index="*")

{'index_1': {'aliases': {}},
 'index_2': {'aliases': {}},
 'index_3': {'aliases': {}},
 '.plugins-ml-config': {'aliases': {}},
 'opensearch-self-query-demo': {'aliases': {}},
 'oci_security': {'aliases': {}},
 'oci': {'aliases': {}},
 'oci_ilm': {'aliases': {}},
 'index_rag': {'aliases': {}},
 'oci4': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}}}

## **Define embedding model**

In [16]:
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Integrate Opensearch as vector db and the HuggingFace embedding model**

In [17]:

vector = OpenSearchVectorSearch(
  embedding_function = embeddings,
  index_name = 'index_rag',
  http_auth = auth,
  use_ssl = True,
  verify_certs = True,
  connection_class = RequestsHttpConnection,
  opensearch_url = "http://" + apiEndpoint
)


retriever = vector.as_retriever()

### **Add custom script to store question in a different index**

## **Add data/vector to the Opensearch cluster**

In [24]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={"year": 1979, "rating": 9.9, "director": "Andrei Tarkovsky", "genre": "science fiction"},
    ),
]

In [26]:
vectorstore = OpenSearchVectorSearch.from_documents(
    docs,
    embeddings,
    index_name="opensearch-self-query-demo",
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    opensearch_url = "http://" + apiEndpoint
)

In [35]:
vectorstore.similarity_search(query = "seems to work? animals", k=2)

[Document(page_content='Toys come alive and have a blast doing so', metadata={'year': 1995, 'genre': 'animated'}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction'}),
 Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3}),
 Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction'})]

## **Full RAG: GenAI**

In [5]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

compartment_id="ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q"

# Use Session Token to authN
llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_id,
    model_kwargs = {"max_tokens":1024},
    verbose=False
)

prompt = PromptTemplate(input_variables=["query"], template="{query}")

llm_chain = LLMChain(llm=llm, prompt=prompt)

response = llm_chain.invoke("what is the capital of france?")
print(response)

{'query': 'what is the capital of france?', 'text': " The capital of France is Paris. Paris is France's largest city and serves as the country's political, cultural, and commercial center. It is located in the north-central part of France, on the River Seine, and is known for its history, architecture, and artistic heritage. \n\nWould you like to know more about Paris? "}


In [ ]:



template = """Answer the question based only on the following context:
{context}
 
Question: {question}
"""
prompt = PromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke("when was oracle founded?"))

### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [18]:
#!pip install opensearch-py

## **Create a notebook session in the same Private Subnet as the OCI Search with OpenSearch cluster**

In [1]:
import requests

In [2]:
searchIndex = 'index_3'
apiEndpoint = "https://amaaaaaangencdyaqhkmbdwh55qlb22dbbouezyxoezj4ivzf4pirlkl6vsa.opensearch.eu-frankfurt-1.oci.oraclecloud.com:9200"
username = "bobpeulen"
password = "xx"

## **Check connection**

In [3]:
def opensearch_check_connection(apiEndpoint, username, password):
    auth=(username, password)
    check_connection = apiEndpoint + '/' + "_cluster/health?pretty"
    headers = {'Content-Type': 'application/x-ndjson'}
    resp = requests.get(check_connection, auth=auth, headers=headers)
    return resp.json()

respx = opensearch_check_connection(apiEndpoint, username, password)
respx

{'cluster_name': 'amaaaaaangencdyaqhkmbdwh55qlb22dbbouezyxoezj4ivzf4pirlkl6vsa',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 2,
 'number_of_data_nodes': 1,
 'discovered_master': True,
 'discovered_cluster_manager': True,
 'active_primary_shards': 9,
 'active_shards': 9,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 5,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 64.28571428571429}

## **Create a new index without schema**

In [21]:
def opensearch_create_index(apiEndpoint, username, password, searchIndex):
    auth=(username, password)
    create_index_url = apiEndpoint + '/' + searchIndex
    headers = {'Content-Type': 'application/x-ndjson'}
    resp = requests.put(create_index_url, auth=auth, headers=headers)
    return resp.json()

respx = opensearch_create_index(apiEndpoint, username, password, searchIndex)
respx

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_3'}

## **Create a new index with schema**

In [8]:
%%writefile mapping.json
{"mappings": {
        "properties": {
            "sensor_id": {"type": "integer"},
            "temperature": {"type": "integer"},
            "humidity": {"type": "integer"},
            "timestamp": {"type": "integer"}
        }
    }
}

Overwriting mapping.json


In [9]:
import json
f = open('mapping.json')

data = json.load(f)
data

{'mappings': {'properties': {'sensor_id': {'type': 'integer'},
   'temperature': {'type': 'integer'},
   'humidity': {'type': 'integer'},
   'timestamp': {'type': 'integer'}}}}

In [10]:
def opensearch_create_index(apiEndpoint, username, password, searchIndex):
    auth=(username, password)
    create_index_url = apiEndpoint + '/' + "test_88"
    headers = {'Content-Type': 'application/x-ndjson'}
    resp = requests.put(create_index_url, auth=auth, headers=headers, data = data)
    return resp.json()

respxxx = opensearch_create_index(apiEndpoint, username, password, searchIndex)
respxxx

{'error': {'root_cause': [{'type': 'not_x_content_exception',
    'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}],
  'type': 'not_x_content_exception',
  'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'},
 'status': 500}

In [ ]:
 -XPUT https://<OpenSearch Endpoint>:<OpenSearch Port>/sensor -H 'Content-Type: application/json' -k -u osadmin:<OpenSearch Admin credential> -d @opensearch-mapping.json
{"acknowledged":true,"shards_acknowledged":true,"index":"sensor"}

## **Insert data in the index**

In [31]:
openSearchData = ""

openSearchData = openSearchData + '{"index":{"_index":"' + searchIndex + '","_id":' + str(1) + '}\n'
openSearchData = openSearchData + '{"sensor_id":' +str(55) + ',"temperature":' +str(22) + ',"humidity":' +str(5) + ',"timestamp":' +str(5165) + '}\n'

In [34]:
def opensearch_insert(apiEndpoint, username, password, searchIndex, openSearchData):
    auth=(username, password)
    bulkinserturl = apiEndpoint + '/' + searchIndex + "/_bulk?pretty"
    headers = {'Content-Type': 'application/x-ndjson'}
    resp = requests.post(bulkinserturl, auth=auth, headers=headers, data=openSearchData)
    return resp.json()

respxx = opensearch_insert(apiEndpoint, username, password, searchIndex, openSearchData)
respxx



{'took': 6,
 'errors': False,
 'items': [{'index': {'_index': 'index_3',
    '_id': '1',
    '_version': 2,
    'result': 'updated',
    '_shards': {'total': 2, 'successful': 1, 'failed': 0},
    '_seq_no': 1,
    '_primary_term': 1,
    'status': 200}}]}

## **Search OpenSearch**